In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import sys
import os
import pandas as pd
import opendatasets as od
import re

if os.getcwd().endswith('notebooks'):
    # Move up one level to the project root
    os.chdir('..') 

print(f"Current Working Directory: {os.getcwd()}")

#Data directory and dataset URL
DATA_DIR = "data"
KAGGLE_DATASET_URL = "https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions"

#Checking data directory existence
os.makedirs(DATA_DIR, exist_ok=True)

#Downloading dataset
od.download(KAGGLE_DATASET_URL, data_dir=DATA_DIR)

Current Working Directory: /home/hsozer/Projects/DeepLearning/recipe_bot
Skipping, found downloaded files in "data/food-com-recipes-and-user-interactions" (use force=True to force download)


In [7]:
from scripts.scraper import sitemap_category_extract, recipe_harvester

# CONFIG
SITEMAP_URL = "https://www.allrecipes.com/sitemap.xml"
CATEGORY_FILTER = "/appetizers-and-snacks/"
RECIPE_PATTERN = re.compile(r'(/recipe/\d+)|(-recipe-\d+/?$)')
EXCLUDE_LIST = ['/gallery/', '/article/']

categories = sitemap_category_extract(SITEMAP_URL, CATEGORY_FILTER)

if categories:
    print(f"\n--- Moving to Harvest Phase ({len(categories)} Categories) ---")

    recipe_url_dataset = recipe_harvester(
        category_urls = categories,
        url_pattern = RECIPE_PATTERN,
        exclude_substrings = EXCLUDE_LIST
    )

    df = pd.DataFrame(recipe_url_dataset, columns=["recipe_url"])
    os.makedirs(DATA_DIR, exist_ok=True)

    output_path = os.path.join(DATA_DIR, "harvested_recipe_urls.csv")
    df.to_csv(output_path, index=False)

    print(f"\n✅ Success! {len(df)} Recipe URLs saved to: {output_path}")

else:
    print("No categories found with the specified filter.")

🗺️  Fetching Sitemap Index...
🔍 Found 4 sub-sitemaps. Scanning for Appetizers and Snacks...
✅ Found 92 URLs matching '/appetizers-and-snacks/'.

--- Moving to Harvest Phase (92 Categories) ---
🚜 Starting Harvest on 92 categories...
📂 Processing category: https://www.allrecipes.com/recipes/17746/appetizers-and-snacks/wraps-and-rolls/lettuce-wraps/
📂 Processing category: https://www.allrecipes.com/recipes/17248/appetizers-and-snacks/dips-and-spreads/salsa/corn-salsa/
📂 Processing category: https://www.allrecipes.com/recipes/1236/appetizers-and-snacks/meat-and-poultry/pork/
📂 Processing category: https://www.allrecipes.com/recipes/23131/appetizers-and-snacks/pastries/empanadas/
📂 Processing category: https://www.allrecipes.com/recipes/116/appetizers-and-snacks/spicy/
📂 Processing category: https://www.allrecipes.com/recipes/17800/appetizers-and-snacks/snacks/energy-balls/
📂 Processing category: https://www.allrecipes.com/recipes/16867/appetizers-and-snacks/snacks/kids/quick-and-easy/
📂 Pr

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import time
import re
import isodate 
import html
import uuid
from fractions import Fraction
import math

# --- CONFIGURATION & CONSTANTS ---
FDA_DAILY_VALUES = {
    'total_fat_pdv': ('total_fat_g', 65.0),
    'sugar_pdv': ('sugar_g', 50.0),
    'sodium_pdv': ('sodium_mg', 2400.0),
    'protein_pdv': ('protein_g', 50.0),
    'sat_fat_pdv': ('sat_fat_g', 20.0),
    'carbs_pdv': ('carbs_g', 300.0)
}

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

TAG_REPLACEMENTS = {
    'snacks': 'snack',
    'appetizers and snacks': 'snack',
    'appetizers': 'appetizer',
    'side dishes': 'side dish',
    'main dish': 'main-dish',
    'main dishes': 'main-dish',
    'desserts': 'dessert',
    'vegetables': 'vegetable',
    'condiments': 'condiment',
    'sauces': 'sauce'
}

# --- HELPER FUNCTIONS (Fraction Conversion) ---
def float_to_cooking_fraction(val):
    """
    Converts float values to cooking fractions.
    e.g, 0.33333 -> 1/3, 1.5 -> 1 1/2, 0.5 -> 1/2
    """
    if val == 0: return "0"
    
    # If very close to an integer, round it
    if abs(val - round(val)) < 0.01:
        return str(round(val))
    
    # Separate integer and decimal parts
    int_part = int(val)
    dec_part = val - int_part
    
    # Limit denominator to standard cooking measures (max 12)
    frac = Fraction(dec_part).limit_denominator(12) 
    
    if frac.denominator == 1: 
        return str(int_part + frac.numerator)
        
    numerator = frac.numerator
    denominator = frac.denominator
    
    if int_part > 0:
        return f"{int_part} {numerator}/{denominator}"
    else:
        return f"{numerator}/{denominator}"

def format_ingredient_string(text):
    """
    Identifies the number at the start of an ingredient string and converts it to a fraction.
    e.g, "0.5 cup sugar" -> "1/2 cup sugar"
    """
    if not isinstance(text, str): return text
    
    # Regex: Capture the leading float or integer
    match = re.match(r"^(\d+(\.\d+)?)", text)
    if match:
        num_str = match.group(1)
        try:
            val = float(num_str)
            formatted_qty = float_to_cooking_fraction(val)
            # Replace only the first occurrence
            return text.replace(num_str, formatted_qty, 1)
        except:
            return text
    return text

# --- SCRAPING & PIPELINE ---
def get_soup(url):
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        if response.status_code == 200:
            return BeautifulSoup(response.content, 'html.parser')
    except Exception as e:
        print("Connection Error")
    return None

def fetch_recipe_data(url):
    soup = get_soup(url)
    if not soup: return None
    result = {'soup': soup, 'json': None}
    
    script = soup.find('script', {'type': 'application/ld+json'})
    if script:
        try:
            data = json.loads(script.string)
            if isinstance(data, list):
                for item in data:
                    if 'Recipe' in item.get('@type', []):
                        result['json'] = item
                        break
            elif isinstance(data, dict):
                 if 'Recipe' in data.get('@type', []): result['json'] = data
        except: pass
    return result

def process_recipes_to_final_format(raw_results_list):
    processed_rows = []
    
    for raw_item in raw_results_list:
        json_data = raw_item.get('json')
        soup = raw_item.get('soup')
        if not json_data: continue 
        
        row = json_data.copy()
        
        # Generate UUID
        row['id'] = str(uuid.uuid4())
        
        # Parse Time
        def parse_minutes(iso_str):
            if not isinstance(iso_str, str): return 0
            try:
                val = int(isodate.parse_duration(iso_str).total_seconds() / 60)
                return val if val < 1440 else 0
            except: return 0
        row['minutes'] = parse_minutes(row.get('totalTime'))

        # Parse Nutrition
        nutrition_map = {
            'calories': ['calories'], 'total_fat_g': ['fatContent'], 'sugar_g': ['sugarContent'],
            'sodium_mg': ['sodiumContent'], 'protein_g': ['proteinContent'], 
            'sat_fat_g': ['saturatedFatContent'], 'carbs_g': ['carbohydrateContent']
        }
        nutri_data = row.get('nutrition', {})
        if isinstance(nutri_data, str):
            import ast
            try: nutri_data = ast.literal_eval(nutri_data)
            except: nutri_data = {}
            
        for target, keys in nutrition_map.items():
            row[target] = 0.0
            for k in keys:
                val = nutri_data.get(k)
                match = re.search(r"(\d+(\.\d+)?)", str(val))
                if match:
                    row[target] = float(match.group(1))
                    break
        
        # Calculate PDV
        for pdv_col, (mass_col, ref_val) in FDA_DAILY_VALUES.items():
            row[pdv_col] = round((row[mass_col] / ref_val) * 100, 1)

        # Generate Tags
        tags = set()
        def clean_and_add_tag(val):
            if pd.isna(val) or val is None or val == "": return
            s_val = str(val).lower().strip()
            if s_val in ["nan", "recipe", "recipes"]: return
            s_val = s_val.replace(" recipes", "").replace(" recipe", "")
            final_tag = TAG_REPLACEMENTS.get(s_val, s_val)
            tags.add(final_tag)

        cats = row.get('recipeCategory')
        if isinstance(cats, list): 
            for c in cats: clean_and_add_tag(c)
        else: clean_and_add_tag(cats)
        
        cuis = row.get('recipeCuisine')
        if isinstance(cuis, list):
            for c in cuis: clean_and_add_tag(c)
        else: clean_and_add_tag(cuis)
        
        if soup:
            breadcrumbs = soup.select('.mntl-breadcrumbs__link')
            for b in breadcrumbs: clean_and_add_tag(b.get_text())

        row['tags'] = list(tags)

        # Cleanup & Format Ingredients
        row['name'] = row.get('name', 'Unknown Recipe')
        if pd.isna(row['name']): row['name'] = 'Unknown'
        
        raw_ingreds = row.get('recipeIngredient', [])
        clean_ingreds = []
        for ing in raw_ingreds:
            clean_ingreds.append(format_ingredient_string(ing))
            
        row['ingredients'] = clean_ingreds
        row['n_ingredients'] = len(clean_ingreds)
        
        # Format Steps
        raw_steps = row.get('recipeInstructions', [])
        clean_steps = []
        if isinstance(raw_steps, list):
            for s in raw_steps:
                if isinstance(s, dict) and 'text' in s: clean_steps.append(s['text'])
                elif isinstance(s, str): clean_steps.append(s)
        row['steps'] = clean_steps
        row['n_steps'] = len(clean_steps)
        
        # Description
        desc = row.get('description', '')
        row['description'] = html.unescape(str(desc)) if desc else ''

        processed_rows.append(row)

    df = pd.DataFrame(processed_rows)
    target_cols = [
        'name', 'id', 'minutes', 'tags', 'n_steps', 'steps', 'description', 
        'ingredients', 'n_ingredients', 'calories', 
        'total_fat_pdv', 'sugar_pdv', 'sodium_pdv', 'protein_pdv', 'sat_fat_pdv', 'carbs_pdv', 
        'total_fat_g', 'sugar_g', 'sodium_mg', 'protein_g', 'sat_fat_g', 'carbs_g'
    ]
    
    if not df.empty:
        for col in target_cols:
            if col not in df.columns: df[col] = None
        return df[target_cols]
    else:
        return pd.DataFrame(columns=target_cols)

# --- EXECUTION ---

df_urls = pd.read_csv("data/harvested_recipe_urls.csv")
target_urls = df_urls['recipe_url'].tolist()
print(f"CSV loaded. Processing URLs. Total:{len(target_urls)}")

raw_results_list = []
print("\n--Starting Scraping & Formatting--")
for i, url in enumerate(target_urls):
    if i % 10 == 0:
        print(f"Processing [{i+1}/{len(target_urls)}] ...")
        
    data = fetch_recipe_data(url)
    if data: 
        raw_results_list.append(data)
    
    time.sleep(1)

if raw_results_list:
    print(f"\nScraping finished. Formatting {len(raw_results_list)} recipes...")
    df_final = process_recipes_to_final_format(raw_results_list)
    
    if not df_final.empty:
        # 1. Save as Pickle (Preserves Python objects like lists)
        pkl_output_path = "data/full_scraped_recipes.pkl"
        df_final.to_pickle(pkl_output_path)
        
        # 2. Save as CSV (Human readable / Excel compatible)
        csv_output_path = "data/full_scraped_recipes.csv"
        df_final.to_csv(csv_output_path, index=False)
        
        print(f"\nSUCCESS! Process Completed.")
        print(f"Total Recipes: {len(df_final)}")
        print(f"Data saved to Pickle: {pkl_output_path}")
        print(f"Data saved to CSV: {csv_output_path}")
        
        # Display first few rows
        display(df_final.head())
    else:
        print("DataFrame is empty.")
else:
    print("No data retrieved.")

In [ ]:
list1 = df_final.columns.tolist()
print(list1)
for x in range(0, 9):
    for i in list1:
        print(df_final.iloc[x][i])
